Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)

Suppose we want to find the options on the SPY. There are two ways to do that:
* The old way by requesting contract details
* The new and faster way

So first the old way. It starts with an ambiguous Option contract and uses that
as a wildcard to get the details of all contracts that match:

** This will take a while **

In [2]:
option = Option('SPY', exchange='SMART')
cds = ib.reqContractDetails(option)

contracts = [cd.summary for cd in cds]

print(len(contracts))
contracts[0]

4752


Contract(conId=178534720, symbol='SPY', secType='OPT', lastTradeDateOrContractMonth='20171215', strike=165.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   171215P00165000', tradingClass='SPY')

So that's a few thousand contracts. Let's put in some restrictions to get this number down:

* Use only the first 3 expirations after today that are on a Friday
* Use only strike prices within +- 20 dollar of the current SPY market price
* Use only strike prices that are a multitude of 5 dollar

For the first restriction the expirations are filtered with an isFriday method,
made unique with set(), then sorted and finally the first 3 taken:

In [3]:
import datetime

def isFriday(date):
    y = int(date[0:4])
    m = int(date[4:6])
    d = int(date[6:8])
    dd = datetime.date(y, m, d)
    return dd.weekday() == 4 and dd > datetime.date.today()

expirations = sorted(set(c.lastTradeDateOrContractMonth for c in contracts
            if isFriday(c.lastTradeDateOrContractMonth)))[:3]

expirations

['20170811', '20170818', '20170825']

Hmmm... perhaps we could have just taken the next three Fridays?
But the number of contracts is going down nicely:

In [4]:
contracts = [c for c in contracts if c.lastTradeDateOrContractMonth in expirations]

print(len(contracts))

718


To get the current price, first create the SPY contract:

In [5]:
spy = Stock('SPY', 'ARCA')

ib.qualifyContracts(spy)

[Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')]

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [6]:
[ticker] = ib.reqTickers(spy)

ticker

Ticker(contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), time=datetime.datetime(2017, 8, 7, 11, 8, 21, 16336, tzinfo=datetime.timezone.utc), bid=247.48, bidSize=2, ask=247.52, askSize=41, last=247.51, lastSize=25, close=247.41)

Apply the final two restrictions:

In [7]:
spyPrice = ticker.marketPrice()

contracts = [c for c in contracts if
        spyPrice - 20 < c.strike < spyPrice + 20 and
        c.strike % 5 == 0]

print(len(contracts))
print(contracts[0])

oldContracts = contracts  # remember for later

48
Contract(conId=273806493, symbol='SPY', secType='OPT', lastTradeDateOrContractMonth='20170818', strike=230.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170818C00230000', tradingClass='SPY')


Finally we have a list of usable option contracts.

Okay so now the new and faster way:

In [8]:
chains = ib.reqSecDefOptParams(spy.symbol, '', spy.secType, spy.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NASDAQOM,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
1,PSE,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
2,BATS,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
3,AMEX,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
4,PEARL,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
5,NASDAQBX,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
6,ISE,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
7,MIAX,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
8,SMART,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."
9,CBOE,756733,SPY,100,"{20170915, 20170830, 20171215, 20170906, 20170...","{10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 3..."


We want the options that trade on SMART:

In [9]:
chain = next(c for c in chains if c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId=756733, tradingClass='SPY', multiplier='100', expirations={'20170915', '20170830', '20171215', '20170906', '20170818', '20170908', '20170929', '20190315', '20170816', '20180119', '20171117', '20171229', '20180329', '20181221', '20170823', '20191220', '20180720', '20170922', '20190118', '20180615', '20180629', '20171020', '20170901', '20170809', '20180921', '20170825', '20180316', '20190621', '20170811'}, strikes={10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 35.0, 254.5, 40.0, 45.0, 246.5, 50.0, 55.0, 60.0, 65.0, 70.0, 247.5, 75.0, 80.0, 85.0, 253.5, 90.0, 95.0, 248.5, 250.5, 249.5, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 15

What we have here is a matrix of expirations x strikes. From this we can build all the contracts:

In [10]:
strikes = [strike for strike in chain.strikes if
        strike % 5 == 0 and
        spyPrice - 20 < strike < spyPrice + 20]
expirations = sorted(exp for exp in chain.expirations if isFriday(exp))[:3]
rights = ['P', 'C']

contracts = [Option('SPY', expiration, strike, right, 'SMART')
        for right in rights for expiration in expirations for strike in strikes]

ib.qualifyContracts(*contracts)

print(len(contracts))
print()
print(contracts[0])

48

Option(conId=281064312, symbol='SPY', lastTradeDateOrContractMonth='20170811', strike=230.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170811P00230000', tradingClass='SPY')


Let's see if the new way ends up with the same contracts as the old way:

In [11]:
set(contracts) == set(oldContracts)

True

Yep. Now to get the market data for all options in one go:

In [12]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Ticker(contract=Option(conId=281064312, symbol='SPY', lastTradeDateOrContractMonth='20170811', strike=230.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170811P00230000', tradingClass='SPY'), time=datetime.datetime(2017, 8, 7, 11, 8, 35, 3975, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.04, modelGreeks=OptionComputation(impliedVol=0.10301014991310896, delta=-5.711513407694938e-12, optPrice=2.3685138096782706e-12, pvDividend=0.0, gamma=1.3636428453469083e-11, vega=2.0545091910513447e-10, theta=-2.3685138096782706e-12, undPrice=247.51000000000002))

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.